In [37]:
from src import Icsr
from src.utils import get_matches

import datasets
import random
from copy import deepcopy
import re

In [38]:
# load matches
dataset = datasets.load_dataset("FAERS-PubMed/BioDEX-ICSR")

Using custom data configuration FAERS-PubMed--BioDEX-ICSR-a832e06da923c0a1
Found cached dataset parquet (/Users/kldooste/.cache/huggingface/datasets/FAERS-PubMed___parquet/FAERS-PubMed--BioDEX-ICSR-a832e06da923c0a1/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 3/3 [00:00<00:00, 302.93it/s]


### Manipulate the full-text
- remove front information
- remove references

In [39]:
def remove_front(text):
    if '==== Body' in text:
        text = ('\n').join(text.split('==== Body')[1:])
    return text.strip()

def remove_refs(text):
    if '==== Refs' in text:
        text = ('\n').join(text.split('==== Refs')[:-1])
    return text.strip() 

def get_fulltext_input(row):
    fulltext_filtered = remove_refs(remove_front(row['fulltext']))
    data = ['\nTITLE:', row['title'], '\nABSTRACT:', row['abstract'], '\nTEXT:', fulltext_filtered]
    return ('\n').join(data).strip()

# numbered_titles_re = r'^(\d[\.\d]*) (.*)\n'
# capitalized_titles_re = r'^([A-Z ]+)\n'

In [40]:
new_dataset = dataset.map(lambda example: {'fulltext_source': get_fulltext_input(example)})

Loading cached processed dataset at /Users/kldooste/.cache/huggingface/datasets/FAERS-PubMed___parquet/FAERS-PubMed--BioDEX-ICSR-a832e06da923c0a1/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-50598b8e576955b6.arrow
100%|██████████| 2407/2407 [00:00<00:00, 3608.24ex/s]


In [42]:
# save to disk
new_dataset.save_to_disk('../data/BioDEX-ICSR-Fulltext')

In [44]:
# load from disk
ds = datasets.load_from_disk('../data/BioDEX-ICSR-Fulltext')